# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [1]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times

import shutil
shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [2]:
from main import Model
import pandas as pd

model = Model(aggregation_name="IMACLIM_format")

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [3]:
#ignore performace warnings
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [4]:
import tqdm

from src.scenarios import DICT_SCENARIOS
scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:00<?, ?it/s]

Problems on sums of columns  Series([], dtype: float64)


100%|██████████| 1/1 [00:55<00:00, 55.82s/it]


## Getting the emissions per types of scenario 

In [42]:
scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"])
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]



for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

  0%|          | 0/7 [00:00<?, ?it/s]

Problems on sums of columns  Series([], dtype: float64)


 14%|█▍        | 1/7 [00:53<05:18, 53.08s/it]

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     186.058374
MDE     Terrestrial transport    2412.591216
RAL     Terrestrial transport     128.913672
dtype: float64


 29%|██▊       | 2/7 [01:45<04:24, 52.94s/it]

...

 71%|███████▏  | 5/7 [04:23<01:45, 52.72s/it]

Problems on sums of columns  region  sector               
AFR     Terrestrial transport      94.507170
MDE     Terrestrial transport    1088.774681
RAL     Terrestrial transport      67.292792
dtype: float64


 86%|████████▌ | 6/7 [05:16<00:52, 52.75s/it]

Problems on sums of columns  Series([], dtype: float64)


100%|██████████| 7/7 [06:10<00:00, 52.86s/it]


In [13]:
import tqdm
from src.utils import footprint_extractor

scenario_list=scenario_list[:1]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in tqdm.tqdm(scenario_list)],
          keys=scenario_list,
          names=("scenario","regions","types"))
end_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in tqdm.tqdm(scenario_list)],
          keys=scenario_list,
          names=("scenario","regions","types"))

end_year_prod=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in tqdm.tqdm(scenario_list)],
          keys=scenario_list,
          names=("scenario","regions","types"))

end_year_cons=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in tqdm.tqdm(scenario_list)],
          keys=scenario_list,
          names=("scenario","regions","types"))

           
pd.concat([start_year,end_year,end_year_prod,end_year_cons],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).loc["INDC"].groupby("types").sum()

100%|██████████| 1/1 [00:00<00:00, 21.77it/s]


,2015,2050,2050-Prod,2050-Cons
,2015,2050,2050,2050
types,,,,
Exportations,-1.043718e+13,-6.763809e+12,-1.896585e+13,-2.095056e+13
Production,4.145956e+13,2.373659e+13,7.457431e+13,7.789206e+13
Importations,1.043718e+13,6.763809e+12,1.896585e+13,2.095056e+13
Consommation,5.524603e+12,5.524603e+12,5.524603e+12,5.524603e+12


In [46]:
Emissions.loc["INDC2020_400"]

2015          2050     2050-Prod     2050-Cons
regions types                                                               
AFR     Exportations -5.869109e+11 -4.317920e+11 -1.857821e+12 -1.347830e+12
        Production    2.412825e+12  1.697926e+12  1.105712e+13  6.910956e+12
        Importations  3.822275e+11  1.788598e+11  8.279193e+11  7.858724e+11
        Consommation  4.259410e+11  4.259410e+11  4.259410e+11  4.259410e+11
BRA     Exportations -2.463252e+11 -1.769440e+11 -5.546541e+11 -5.463641e+11
        Production    1.085191e+12  7.254011e+11  2.119124e+12  2.125627e+12
        Importations  1.788636e+11  1.050770e+11  2.889377e+11  3.025654e+11
        Consommation  9.061388e+10  9.061388e+10  9.061388e+10  9.061388e+10
CAN     Exportations -3.098063e+11 -1.686158e+11 -5.402869e+11 -4.455229e+11
        Production    6.529151e+11  2.943742e+11  8.337175e+11  8.707679e+11
        Importations  2.265260e+11  1.032200e+11  2.349277e+11  3.490324e+11
        Consommation  1.296971e+11  1.296971e+11  1.296971e+11  1.296971e+11
CHN     Exportations -2.311067e+12 -5.790299e+11 -7.877648e+11 -1.816962e+12
        Production    1.151045e+13  2.626045e+12  6.698149e+12  8.892034e+12
        Importations  1.378427e+12  7.850620e+11  2.580024e+12  2.713400e+12
        Consommation  5.477262e+11  5.477262e+11  5.477262e+11  5.477262e+11
CIS     Exportations -7.353366e+11 -3.695373e+11 -8.188040e+11 -1.100330e+12
        Production    1.846605e+12  7.741604e+11  2.127613e+12  2.352896e+12
        Importations  1.618188e+11  8.030071e+10  2.190895e+11  2.485818e+11
        Consommation  2.497628e+11  2.497628e+11  2.497628e+11  2.497628e+11
EUR     Exportations -9.475107e+11 -4.385118e+11 -1.143255e+12 -1.422040e+12
        Production    5.021247e+12  1.977457e+12  3.791803e+12  5.429529e+12
        Importations  2.189241e+12  1.073366e+12  2.775661e+12  2.795099e+12
        Consommation  9.586125e+11  9.586125e+11  9.586125e+11  9.586125e+11
IND     Exportations -4.568872e+11 -2.155643e+11 -9.100936e+11 -7.049599e+11
        Production    3.013071e+12  1.344508e+12  5.974315e+12  5.623335e+12
        Importations  4.362366e+11  2.294510e+11  1.090832e+12  9.996301e+11
        Consommation  2.806296e+11  2.806296e+11  2.806296e+11  2.806296e+11
JAN     Exportations -4.361290e+11 -2.123701e+11 -5.978846e+11 -7.167531e+11
        Production    1.867037e+12  6.694314e+11  1.218967e+12  1.734144e+12
        Importations  6.986244e+11  3.028500e+11  5.998534e+11  6.741260e+11
        Consommation  2.243538e+11  2.243538e+11  2.243538e+11  2.243538e+11
MDE     Exportations -1.178884e+12 -6.023831e+11 -2.061326e+12 -1.971288e+12
        Production    2.579607e+12  1.240212e+12  3.275994e+12  4.453460e+12
        Importations  6.459012e+11  3.032752e+11  9.020304e+11  1.180224e+12
        Consommation  3.319888e+11  3.319888e+11  3.319888e+11  3.319888e+11
RAL     Exportations -6.164752e+11 -4.056155e+11 -1.027384e+12 -1.166440e+12
        Production    1.958445e+12  1.136008e+12  2.753663e+12  3.107966e+12
        Importations  7.291932e+11  3.025014e+11  7.131920e+11  8.041085e+11
        Consommation  2.830767e+11  2.830767e+11  2.830767e+11  2.830767e+11
RAS     Exportations -1.883108e+12 -1.011330e+12 -3.219565e+12 -3.092061e+12
        Production    4.682540e+12  2.206872e+12  7.657379e+12  7.733337e+12
        Importations  1.546373e+12  6.327755e+11  2.084575e+12  2.456531e+12
        Consommation  4.551369e+11  4.551369e+11  4.551369e+11  4.551369e+11
USA     Exportations -7.287437e+11 -3.380189e+11 -9.340081e+11 -1.044697e+12
        Production    4.829634e+12  1.606888e+12  3.165413e+12  4.118153e+12
        Importations  1.863752e+12  8.529744e+11  2.135806e+12  2.066076e+12
        Consommation  1.547064e+12  1.547064e+12  1.547064e+12  1.547064e+12

In [16]:
model.iot.s

In [50]:
model.iot.stressor_extension.F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     3.116139e+11   
CH4                                     9.871424e+10   
N2O                                     1.451475e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            2.419435e+07                                     
CH4                                            2.867235e+05                                     
N2O                                            2.226237e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     6.451246e+08   
CH4                                     3.775609e+08   
N2O                                     5.090450e+07   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     8.477625e+10   
CH4                                     3.905620e+09   
N2O                                     1.695344e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            3.837461e+07                                     
CH4                                            4.732580e+06                                     
N2O                                            5.219661e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [25]:
model.iot.stressor_extension.S.dot(model.iot.Y)

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     2.785334e+11   
CH4                                     3.480231e+11   
N2O                                     1.071688e+11   
SF6                                     8.037921e+08   
HFC                                     2.555006e+09   
PFC                                     8.225578e+08   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            4.787910e+09                                     
CH4                                            6.822239e+09                                     
N2O                                            1.389551e+09                                     
SF6                                            6.785561e+06                                     
HFC                                            2.639568e+07                                     
PFC                                            1.597531e+06                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     1.538574e+10   
CH4                                     2.228198e+10   
N2O                                     4.292760e+09   
SF6                                     2.793052e+07   
HFC                                     9.174403e+07   
PFC                                     1.506917e+07   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                       9.877345e+10           2.866982e+09   
CH4                       7.166073e+10           3.750995e+09   
N2O                       1.130743e+10           3.752828e+08   
SF6                       3.010710e+08           7.187029e+06   
HFC                       1.012377e+09           1.426768e+07   
PFC                       3.104964e+08           7.797248e+06   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2              3.800544e+09                  0.0   
CH4              3.977167e+07                  0.0   
N2O              1.688590e+07                  0.0   
SF6              7.980692e+04                  0.0   
HFC              1.516459e+05                  0.0   
PFC              4.005660e+03                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     1.143294e+11   
CH4                                     1.274554e+11   
N2O                                     3.447924e+10   
SF6                                     1.092944e+08   
HFC                                     2.041571e+09   
PFC                                     8.751837e+08   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.221015e+09                                     
CH4                                            1.512048e+07                                     
N2O                                            1.000969e+07                                     
SF6                                            8.362872e+04                                     
HFC                                            2.104700e+06                                     
PFC                                            3.774773e+04                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

AttributeError: 'IOSystem' object has no attribute 'stressor_extension'

In [51]:
scenario="INDC"
model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)

Problems on sums of columns  Series([], dtype: float64)


In [58]:
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)
    

In [59]:
from pymrio.tools.iomath import calc_F_Y
F_Y=calc_F_Y(model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.stressor_extension.S_Y,model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.Y.sum())

In [60]:
F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     1.369166e+12   
CH4                                     4.337297e+11   
N2O                                     6.377476e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.063049e+08                                     
CH4                                            1.259803e+06                                     
N2O                                            9.781622e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     2.834543e+09   
CH4                                     1.658924e+09   
N2O                                     2.236637e+08   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     2.441102e+11   
CH4                                     1.124609e+10   
N2O                                     4.881682e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.104983e+08                                     
CH4                                            1.362729e+07                                     
N2O                                            1.502983e+05                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [57]:
model.counterfactuals["IMACLIM_all_2050_INDC"].iot.stressor_extension.F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     3.116139e+11   
CH4                                     9.871424e+10   
N2O                                     1.451475e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            2.419435e+07                                     
CH4                                            2.867235e+05                                     
N2O                                            2.226237e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     6.451246e+08   
CH4                                     3.775609e+08   
N2O                                     5.090450e+07   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     8.477625e+10   
CH4                                     3.905620e+09   
N2O                                     1.695344e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            3.837461e+07                                     
CH4                                            4.732580e+06                                     
N2O                                            5.219661e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    